In [ ]:
from sklearn import model_selection
from tweepy import Stream
from tweepy import OAuthHandler
import json
from tweepy.streaming import StreamListener
import nltk
from nltk.corpus import stopwords
import re
import pickle
from sklearn.externals import joblib


#nltk.download('stopwords')

ckey='Hk6d52Oi7VtV2BdXkqfhHiRPz'
csecret='zqxOsWZcdow4kpQAEwE28PiIJIsBOqarMLQ1idg2AVoDGJxnKz'
atoken='2914530456-hOKF00XjFw72cfovacKrq8vkmWMjjFDt12wCNxo'
asecret='CivraKC9GwvxjFiGBa4KCfaA3ISPSiIqGAmtYdxTkvBjO'


# load the model from disk
loaded_model = joblib.load('hybrid_model.sav')

REPLACE_BY_SPACE_RE = re.compile('[/(){}\\[\\]\\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
        

def clean_text(text):
    if type(text) is str:
        text = text.lower()
        text = re.sub("[^a-zA-Z]", "/,",str(text))
        text = REPLACE_BY_SPACE_RE.sub(' ', text)
        text = BAD_SYMBOLS_RE.sub('', text)
        text = ' '.join(word for word in text.split() if word not in STOPWORDS)
        return text


class listener(StreamListener):
    def on_data(self,data):
        all_data = json.loads(data)
        tweet = all_data["text"]
        #print(clean_text(tweet)+"\n\n")
        tweet01 = clean_text(tweet)
        result = loaded_model.predict([tweet01])
        print(str(tweet01)+"\t\t"+str(result))
        return True
    
    def on_error(self, status):
        print(status)
         
auth=OAuthHandler(ckey,csecret)
auth.set_access_token(atoken,asecret)
twitterStream=Stream(auth,listener())
twitterStream.filter(track=["bitcoin"])